# `SELECT`

Nesta aula vamos explorar mais sobre o comando `SELECT`, usado para consultar a base de dados.

Continuaremos a utilizar a mesma base da Aula 02. Se não tiver, instale a base de teste **musica** usando o script `musica.sql`. Este script instala a base de dados de exemplo do livro-texto da disciplina, de modo que vocês podem praticar com os exercícios do livro também!

Eis aqui o diagrama do modelo relacional dessa base de dados:

<center><img src="imgs/modelo_relacional.png"/></center>

Eis a lista completa de chaves primárias e estrangeiras:

```SQL
ALTER TABLE AUTOR
    ADD PRIMARY KEY (Codigo_Autor);

ALTER TABLE CD
    ADD PRIMARY KEY (Codigo_CD) ;

ALTER TABLE GRAVADORA
    ADD PRIMARY KEY (Codigo_Gravadora) ;

ALTER TABLE FAIXA
    ADD PRIMARY KEY (Codigo_Musica, Codigo_CD);

ALTER TABLE MUSICA
    ADD PRIMARY KEY (Codigo_Musica) ;

ALTER TABLE MUSICA_AUTOR
    ADD PRIMARY KEY (Codigo_Musica, Codigo_Autor);

ALTER TABLE CD
    ADD FOREIGN KEY (Codigo_Gravadora)
        REFERENCES GRAVADORA(Codigo_Gravadora);

ALTER TABLE CD
    ADD FOREIGN KEY (CD_Indicado)
        REFERENCES CD(Codigo_CD);

ALTER TABLE FAIXA
    ADD FOREIGN KEY (Codigo_CD)
        REFERENCES CD(Codigo_CD);

ALTER TABLE FAIXA
    ADD FOREIGN KEY (Codigo_Musica)
        REFERENCES MUSICA(Codigo_Musica);

ALTER TABLE MUSICA_AUTOR
    ADD FOREIGN KEY (Codigo_Autor)
        REFERENCES AUTOR(Codigo_Autor);

ALTER TABLE MUSICA_AUTOR
    ADD FOREIGN KEY (Codigo_Musica)
        REFERENCES MUSICA(Codigo_Musica);
```

**Atividade:** Quais destas tabelas são tabelas de relacionamento?

<div class="alert alert-success">

Sua resposta aqui! Dê dois cliques e edite.

</div>

## Criar conexão com o banco de dados
Vamos agora criar a conexão com o banco de dados e o objeto auxiliar de conexão. Aqui resolvi usar uma estratégia diferente de implementação que dá o mesmo resultado, para deixar de exemplo para vocês:

In [ ]:
# !pip install mysql-connector-python

In [27]:
from functools import partial
from dotenv import load_dotenv
import insperautograder.jupyter as ia
import mysql.connector
import os

load_dotenv(override=True)

connection = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    database="musica"
)

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print("Executando query:")
        cursor.execute(query, args)
        for result in cursor:
            print(result)


db = partial(run_db_query, connection)

# <span style='color:blue;'> Relembrando `SELECT`</span>

As cláusulas das próximas seções já foram vistas nas primeiras aulas (Select). Deixamos aqui para que relembrem, agora utilizando a chamada `db()` direto no python!

## Explorando a estrutura da base de dados

Agora podemos usar nosso helper para enviar comandos à base de dados. Vamos ver quais tabelas existem na base 'musica':

In [2]:
db("SHOW TABLES")

Executando query:
('autor',)
('cd',)
('cd_categoria',)
('faixa',)
('gravadora',)
('musica',)
('musica_autor',)


Para saber qual o schema da tabela 'cd', podemos usar o comando '`DESCRIBE`'

In [3]:
db("DESCRIBE CD")

Executando query:
('Codigo_CD', 'int', 'NO', 'PRI', None, '')
('Codigo_Gravadora', 'int', 'YES', 'MUL', None, '')
('Nome_CD', 'varchar(60)', 'YES', '', None, '')
('Preco_Venda', 'int', 'YES', '', None, '')
('Data_Lancamento', 'date', 'YES', '', None, '')
('CD_Indicado', 'int', 'YES', 'MUL', None, '')


## Consultando a base de dados

Relembrando: vamos usar o comando '`SELECT`' para listar os conteudos da tabela 'cd'

In [4]:
db("SELECT * FROM CD")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


O comando acima lista todos os registros da tabela 'cd', com todas as colunas presentes:

![Seleção da tabela inteira](imgs/tudo.png)

Vamos agora selecionar apenas algumas colunas para exibir.

In [5]:
db("SELECT Nome_CD, Data_Lancamento FROM CD")

Executando query:
('Mais do Mesmo', datetime.date(1998, 10, 1))
('Bate-Boca', datetime.date(1999, 7, 1))
('Elis Regina - Essa Mulher', datetime.date(1989, 5, 1))
('A Força que nunca Seca', datetime.date(1998, 12, 1))
('Perfil', datetime.date(2001, 5, 1))
('Barry Manilow Greatest Hits Vol I', datetime.date(1991, 11, 1))
('Listen Without Prejudice', datetime.date(1991, 10, 1))


Agora vemos apenas as colunas escolhidas. Relembrando: a operação de seleção de colunas chama-se **projeção**.

![Projeção](imgs/projecao.png)

Vamos agora atuar na escolha de linhas, selecionando quais desejamos. Para escolher todas as linhas cujo Nome_CD terminem em 'a', podemos executar a query a seguir:

In [6]:
db("SELECT * FROM CD WHERE Nome_CD LIKE '%a'")

Executando query:
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)


Formatar a query facilita sua compreensão!

In [7]:
sql = """
SELECT 
    *
FROM
    CD
WHERE
    Nome_CD LIKE '%a'
"""
db(sql)

Executando query:
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)


Vemos apenas as linhas escolhidas. Recordando: a operação de filtragem de linhas apropriadas chama-se **seleção**.

![Seleção](imgs/selecao.png)

### Comentários

Comentários podem ser úteis para auxiliar na compreensão das queries. Em SQL, utilizaremos `--` **seguido de um espaço** e tudo a direita será considerado como comentário. Veja um exemplo:

In [8]:
sql = """
SELECT 
    * 
FROM
    CD
WHERE
    Nome_CD LIKE '%a' -- Aqui, % tem o mesmo significado que asterisco em uma busca no terminal!
"""
db(sql)

Executando query:
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)


## Cláusula `WHERE`

A cláusula `WHERE` permite **filtrar as linhas** da tabela (ou tabelas - mais sobre *JOIN* daqui a pouco). Basta especificar a condição de filtragem: o resultado da query será o conjunto de linhas para as quais a condição é verdadeira.

Já vimos alguns usos da cláusula `WHERE` acima. Vamos ver mais exemplos:

In [9]:
# Queries equivalentes.
db("SELECT * FROM CD")
db("SELECT * FROM CD WHERE True")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)
Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None)


Qual o cd mais antigo que custa 13 reais ou mais?

In [10]:
db("""
SELECT 
    Nome_CD, Preco_Venda, Data_Lancamento
FROM 
    CD 
WHERE 
    Preco_Venda >= 13 
""")

Executando query:
('Mais do Mesmo', 15, datetime.date(1998, 10, 1))
('Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1))
('A Força que nunca Seca', 14, datetime.date(1998, 12, 1))


Perceba que, com os resultados obtidos, podemos comparar as datas uma a uma e encontrar a mais antiga. Logo, obtemos o nome do cd mais antigo que custa 13 reais ou mais! Mas será que existe forma mais fácil?

## Ordenar linhas

Podemos usar o `ORDER BY` para indicar por qual coluna queremos fazer a ordenação. Podemos também indicar com `ASC` que a ordenação será crescente (padrão) e `DESC` para decrescente.

In [11]:
db("""
SELECT
    Nome_CD, Preco_Venda, Data_Lancamento
FROM 
    CD 
WHERE 
    Preco_Venda >= 13
ORDER BY 
    Data_Lancamento DESC
""")

Executando query:
('A Força que nunca Seca', 14, datetime.date(1998, 12, 1))
('Mais do Mesmo', 15, datetime.date(1998, 10, 1))
('Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1))


Pronto! Agora o cd requisitado está na última linha, muito mais fácil, não?!

Podemos indicar apenas um inteiro como id da coluna a ser ordenada. Entretanto, é uma boa prática escrever o nome da coluna, tanto por facilidade de leitura quanto por mudanças que podem ser feitas nas colunas retornadas pela query, sendo fácil esquecer ou perceber que o id mudou.

In [12]:
db("""
SELECT 
    Nome_CD, Preco_Venda, Data_Lancamento
FROM 
    CD 
WHERE 
    Preco_Venda >= 13
ORDER BY 
    3 DESC
""")

Executando query:
('A Força que nunca Seca', 14, datetime.date(1998, 12, 1))
('Mais do Mesmo', 15, datetime.date(1998, 10, 1))
('Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1))


Mas será que precisamos retornar todas as linhas? O nosso interesse está em apenas uma (a que possui **o cd mais antigo que custa 13 reais ou mais**).

### `LIMIT`ar a quantidade de linhas retornadas!

Podemos utilizar o `LIMIT` para definir um limite máximo da quantidade de linhas retornadas. Isto será bastante útil quando apenas estivermos explorando as tabelas, quando ver três ou cinco linhas da tabela é suficiente (semelhante ao `dataframe.head()` do `pandas`).

Vamos tentar?

In [13]:
db("""
SELECT 
    Nome_CD, Preco_Venda, Data_Lancamento
FROM 
    CD 
WHERE 
    Preco_Venda >= 13
ORDER BY 
    Data_Lancamento DESC
LIMIT 1
""")

Executando query:
('A Força que nunca Seca', 14, datetime.date(1998, 12, 1))


# <span style='color:blue;'> Conteúdo novo!</span>
## Agregação

Agregação em **SQL** refere-se ao processo de **combinar e resumir dados** para obter informações mais significativas.

Utilizaremos funções como `SUM`, `COUNT`, `AVG`, `MAX` e `MIN`, que **operam em conjuntos de registros** para gerar resultados consolidados.

**Exemplo**: Quantas músicas tem na base?

In [14]:
db("SELECT COUNT(*) FROM MUSICA")

Executando query:
(88,)


Note o comando `COUNT` acima. Trata-se de um comando de *agregação*: uma operação que trabalha em cima de todo o resultado da query e retorna um valor agregado para esses resultados. 

Usamos o argumento "\*" para indicar que queremos apenas contar quantas linhas não são nulas. Normalmente é melhor especificar direito a coluna sobre a qual queremos operar.

Por exemplo: qual a duração da musica mais longa?

In [16]:
db("SELECT MAX(duracao) FROM MUSICA")

Executando query:
(9,)


**Exercícios para entrega**

Esta aula tem atividade para entrega, confira os prazos e exercícios

In [17]:
ia.tasks()

|    | Atividade   | De                  | Até                 |
|---:|:------------|:--------------------|:--------------------|
|  0 | newborn     | 2025-02-01 00:00:00 | 2025-05-30 00:00:00 |
|  1 | select01    | 2025-02-05 00:00:00 | 2025-02-15 23:59:59 |
|  2 | ddl         | 2025-02-19 00:00:00 | 2025-02-26 23:59:59 |
|  3 | dml         | 2025-02-24 00:00:00 | 2025-03-09 23:59:59 |
|  4 | agg_join    | 2025-02-26 00:00:00 | 2025-03-09 23:59:59 |

In [29]:
ia.grades(task="agg_join")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | agg_join    | ex01        |      1 |      0 |
|  1 | agg_join    | ex02        |      1 |      0 |
|  2 | agg_join    | ex03        |      1 |      0 |
|  3 | agg_join    | ex04        |      1 |      0 |
|  4 | agg_join    | ex05        |      1 |      0 |
|  5 | agg_join    | ex06        |      1 |      0 |

**Exercício 1:** Quantas musicas foram escritas pelo autor número 1 (Renato Russo)?

**Obs**: o atributo retornado de ve se chamar `qt_musicas`

In [38]:
sql_ex01 = """
SELECT COUNT(*) as qt_musicas FROM MUSICA_AUTOR WHERE Codigo_Autor = 1;
"""

db(sql_ex01)

Executando query:
(15,)


In [39]:
ia.sender(answer="sql_ex01", task="agg_join", question="ex01", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex01', style=ButtonStyle()), Output()), _dom_classes=('widget…

**Exercício 2:** Se eu fosse comprar uma cópia de cada CD da base, quanto gastaria?

**Obs**: o atributo retornado de ve se chamar `valor_total`

In [40]:
sql_ex02 = """
SELECT SUM(preco_venda) as valor_total FROM CD;
"""

db(sql_ex02)

Executando query:
(Decimal('84'),)


In [41]:
ia.sender(answer="sql_ex02", task="agg_join", question="ex02", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex02', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Variáveis

Verificando manualmente a tabela `musica` podemos encontrar o nome da musica mais longa:

In [25]:
db("SELECT * from MUSICA")

Executando query:
(1, 'Será', 2)
(2, 'Ainda é Cedo', 4)
(3, 'Geração Coca-Cola', 2)
(4, 'Eduardo e Monica', 4)
(5, 'Tempo Perdido', 5)
(6, 'Índios', 4)
(7, 'Que País é Este', 3)
(8, 'Faroeste Caboclo', 9)
(9, 'Há Tempos', 3)
(10, 'Pais e Filhos', 5)
(11, 'Meninos e Meninas', 3)
(12, 'Vento no Litoral', 6)
(13, 'Perfeição', 4)
(14, 'Giz', 3)
(15, 'Dezesseis', 5)
(16, 'Antes das Seis', 3)
(17, 'Meninos, Eu Vi', 3)
(18, 'Eu Te Amo', 3)
(19, 'Piano na Mangueira', 2)
(20, 'A Violeira', 3)
(21, 'Anos Dourados', 3)
(22, 'Olha, Maria', 4)
(23, 'Biscate', 3)
(24, 'Retrato em Preto e Branco', 3)
(25, 'Falando de Amor', 3)
(26, 'Pois É', 2)
(27, 'Noite dos Mascarados', 2)
(28, 'Sabiá', 3)
(29, 'Imagina', 3)
(30, 'Bate-Boca', 4)
(31, 'Cai Dentro', 2)
(32, 'O Bêbado e o Equilibrista', 3)
(33, 'Essa Mulher', 3)
(34, 'Basta de Clamares Inocência', 3)
(35, 'Beguine Dodói', 2)
(36, 'Eu hein Rosa', 3)
(37, 'Altos e Baixos', 3)
(38, 'Bolero de Satã', 3)
(39, 'Pé Sem Cabeça', 3)
(40, 'As Aparências Engana

Nenhuma surpresa aqui: é "Faroeste Caboclo"...

Agora suponha que eu quero localizar o *nome* da música mais comprida. Note que o comando a seguir NÃO FAZ SENTIDO:

In [26]:
db("SELECT Nome_musica, MAX(duracao) FROM MUSICA")

Executando query:


ProgrammingError: 1140 (42000): In aggregated query without GROUP BY, expression #1 of SELECT list contains nonaggregated column 'musica.MUSICA.Nome_Musica'; this is incompatible with sql_mode=only_full_group_by

A resposta está errada. Nem poderia estar certa: e se tivermos mais de uma música com a duração máxima?

Talvez se especificássemos que a duracao da música tem que coincidir com a duracao máxima...

In [30]:
try:
    db("SELECT Nome_musica FROM MUSICA WHERE duracao=MAX(duracao)")
except mysql.connector.DatabaseError as e:
    print(f"DatabaseError: {e}")

Executando query:
DatabaseError: 1111 (HY000): Invalid use of group function


Também não funciona: não podemos usar uma função de agrupamento dentro da cláusula `WHERE`. Como resolver isso então?

Primeiro descubra qual é a maior duração e guarde em uma *variável* ***no banco de dados***.

In [31]:
db("SELECT MAX(duracao) INTO @max_duracao from MUSICA")

Executando query:


Podemos checar o valor dessa variavel agora:

In [42]:
db("SELECT @max_duracao")

Executando query:
(9,)


Agora podemos terminar o serviço!

In [43]:
db("SELECT Nome_musica FROM MUSICA WHERE duracao=@max_duracao")

Executando query:
('Faroeste Caboclo',)


**Exercício 3**: Encontre todas as musicas com duração acima da média.

Você deve criar uma variável chamada `avg_duracao` e utilizá-la em sua solução!

Ainda, retorne a `duracao` e o `Nome_musica`, ordenando de forma decrescente pela `duracao`.

In [45]:
sql_ex03_part1 = """
SELECT AVG(duracao) INTO @avg_duracao FROM MUSICA;
"""

sql_ex03_part2 = """
SELECT @avg_duracao;
"""

sql_ex03_part3 = """
SELECT duracao, Nome_musica FROM MUSICA WHERE duracao > @avg_duracao ORDER BY duracao DESC;
"""

# Para envio ao servidor: Não alterar
sql_ex03 = f"""
{sql_ex03_part1.replace(';','').strip()};
{sql_ex03_part2.replace(';','').strip()};
{sql_ex03_part3.replace(';','').strip()};
"""

print("-> Criar variável:")
db(sql_ex03_part1)
print("\n-> Selecionar variável:")
db(sql_ex03_part2)
print("\n-> Músicas com duração acima da média:")
db(sql_ex03_part3)

-> Criar variável:
Executando query:

-> Selecionar variável:
Executando query:
(Decimal('3.284090909'),)

-> Músicas com duração acima da média:
Executando query:
(9, 'Faroeste Caboclo')
(6, 'Vento no Litoral')
(5, 'Tempo Perdido')
(5, 'Pais e Filhos')
(5, 'Dezesseis')
(4, 'Ainda é Cedo')
(4, 'Eduardo e Monica')
(4, 'Índios')
(4, 'Perfeição')
(4, 'Olha, Maria')
(4, 'Bate-Boca')
(4, 'As Aparências Enganam')
(4, 'É o Amor')
(4, 'Devolva-me')
(4, 'Inverno')
(4, 'Vambora')
(4, 'Maresia')
(4, 'Naquela Estação')
(4, 'New York City Rhythm')
(4, "It's a Miracle")
(4, 'Trying to get the feeling again')
(4, 'Some Kind of Friend')
(4, 'Praying for Time')
(4, 'Freedom 90')
(4, 'Something to Save')
(4, 'Cowboys and Angels')


In [46]:
ia.sender(answer="sql_ex03", task="agg_join", question="ex03", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03', style=ButtonStyle()), Output()), _dom_classes=('widget…

## Pesquisa em múltiplas tabelas

### Produto cartesiano

Quando listamos várias tabelas no comando `SELECT` temos como resultado o produto cartesiano destas. 

(Para os matemáticos: não se trata estritamente do produto cartesiano, pois não geramos um conjunto de duplas de tuplas. Geramos um "aplainamento" - *flattening* - do resultado, para cada linha.)

Por exemplo, se temos uma tabela com 2 linhas e outra com 3 linhas, o resultado terá 6 linhas, como ilustrado a seguir:

```SQL
DROP DATABASE IF EXISTS test;
CREATE DATABASE test;
USE test;

CREATE TABLE tab1 (
    coluna11 INT,
    coluna21 INT
);

CREATE TABLE tab2 (
    coluna12 INT,
    coluna22 INT
);

INSERT INTO tab1 VALUES (1, 2), (3, 4);
INSERT INTO tab2 VALUES (1000, 2000), (3000, 4000), (5000, 6000);

SELECT * FROM tab1;

SELECT * FROM tab2;

SELECT * FROM tab1, tab2;
```

Resultado:

![Produto cartesiano](imgs/cartesiano.png)

Observe o que acontece quando unimos as tabelas *cd* e *gravadora*

In [47]:
db("SELECT * FROM CD, GRAVADORA")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 'EPIC', None, None, 'PAULO', 'www.epic.com.br')
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 4, 'EPIC', None, None, 'PAULO', 'www.epic.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.c

### Inner join

![](imgs/inner_join.png)

Fonte da imagem: https://blog.codinghorror.com/content/images/uploads/2007/10/6a0120a85dcdae970b012877702708970c-pi.png

E se filtrássemos o resultado anterior, mantendo apenas as linhas em que o código de gravadora na tabela *'cd'* coincida com o código de gravadora na tabela *'gravadora'*?

In [48]:
db("""
SELECT
    * 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
""")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 1, 'EMI', 'Rod. Pres. Dutra, s/n km 229,8', None, 'JOÃO', 'www.emi-music.com.br')
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2, 3, 'SOM LIVRE', None, None, 'MARTA', 'www.somlivre.com.br')
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None, 2, 'BMG', 'Av. Piramboia, 2898 - Parte 7', None, 'MARIA', 'www.bmg.com.br')


In [49]:
db("""
SELECT
    Nome_CD, Nome_Gravadora 
FROM
    CD as c, GRAVADORA as g 
WHERE
    c.Codigo_Gravadora = g.Codigo_Gravadora
""")

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


Opa, parece que com isso conseguimos associar a gravadora ao cd!

Estamos mantendo apenas as linhas do produto cartesiano onde o codigo da gravadora declarado na tabela cd bate com o codigo da gravadora declarado na tabela gravadora. Logo, nestas linhas, a informação proveniente da tabela cd e a informação proveniente da tabela gravadora estarão se referindo à mesma gravadora. Conseguimos, portanto, conectar as informações de cada CD com as informações da gravadora à qual aquele CD pertence.

Denominamos *"inner join"*, *"equi-join"* ou "união regular" a essa operação de junção de tabelas através da união de **chave primária** de uma com **chave estrangeira** de outra. Eu chamo informalmente de "vamos bater chaves!"

Vamos construir uma lista de cds e suas respectivas gravadoras:

In [50]:
db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, GRAVADORA as g 
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
""")

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


Vamos aprimorar nossa query para procurar apenas os CDs gravados pela 'SOM LIVRE':

In [51]:
db("""
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD as c, 
    GRAVADORA as g
WHERE 
    c.Codigo_Gravadora = g.Codigo_Gravadora
    AND g.Nome_Gravadora = 'SOM LIVRE'
""")

Executando query:
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('Perfil', 'SOM LIVRE')


Existem "apelidos" para a união regular, que produzem o mesmo resultado:

In [52]:
db("""
SELECT 
    Nome_CD, 
    Nome_Gravadora 
FROM 
    CD
    INNER JOIN GRAVADORA USING (Codigo_Gravadora)
""")

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


In [53]:
db("""
SELECT
    Nome_CD,
    Nome_Gravadora
FROM
    CD as c
    INNER JOIN GRAVADORA as g ON c.Codigo_Gravadora = g.Codigo_Gravadora
""")

Executando query:
('Mais do Mesmo', 'EMI')
('Bate-Boca', 'BMG')
('Elis Regina - Essa Mulher', 'SOM LIVRE')
('A Força que nunca Seca', 'BMG')
('Perfil', 'SOM LIVRE')
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Listen Without Prejudice', 'BMG')


**Exercício 4**: Construa a lista das musicas do Renato Russo. Sua query deverá filtrar utilizando o texto `"Renato Russo"`.

- **Dica**: Reveja os relacionamentos!

In [57]:
sql_ex04 = """
SELECT
	Nome_Musica
FROM
	AUTOR INNER JOIN MUSICA_AUTOR USING (Codigo_Autor)
    INNER JOIN MUSICA USING(Codigo_Musica)
WHERE
	Nome_Autor = 'Renato Russo';
"""

db(sql_ex04)

Executando query:
('Será',)
('Geração Coca-Cola',)
('Eduardo e Monica',)
('Tempo Perdido',)
('Índios',)
('Que País é Este',)
('Faroeste Caboclo',)
('Há Tempos',)
('Pais e Filhos',)
('Meninos e Meninas',)
('Vento no Litoral',)
('Perfeição',)
('Giz',)
('Dezesseis',)
('Antes das Seis',)


In [58]:
ia.sender(answer="sql_ex04", task="agg_join", question="ex04", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04', style=ButtonStyle()), Output()), _dom_classes=('widget…

### Self join

![](imgs/self_join.webp)

Fonte da imagem: https://cdn.sanity.io/images/oaglaatp/production/8da7e1428ceeeb20c69a8f1f869a055ab1636a9c-3720x2160.png?w=3720&h=2160&auto=format

E se quisermos listar os nomes de CDs e seus respectivos CDs indicados? Veja a solução abaixo. Note o uso de dois *aliases* diferentes para a tabela cd.

In [59]:
# Observe também o uso da função CONCAT(), só para ficar mais bonito!
db("""
SELECT
    CONCAT('"', c1.Nome_CD, '" indicado por "', c2.Nome_CD, '"') AS indicacao
FROM
    CD c1,
    CD c2
WHERE
    c1.Codigo_CD = c2.CD_Indicado
""")

Executando query:
('"Perfil" indicado por "Mais do Mesmo"',)
('"Elis Regina - Essa Mulher" indicado por "Bate-Boca"',)
('"Mais do Mesmo" indicado por "Elis Regina - Essa Mulher"',)
('"Mais do Mesmo" indicado por "A Força que nunca Seca"',)
('"Bate-Boca" indicado por "Perfil"',)
('"Listen Without Prejudice" indicado por "Barry Manilow Greatest Hits Vol I"',)


### *'Non-equi-join'*

Chamamos de *'non-equi-join'* a uma junção de tabelas que não usa a igualdade de colunas como critério de junção. 

Podemos fazer muitas investigações úteis com junção de tabelas. Vamos acompanhar um exemplo (também está no livro-texto).

Na base 'musica' temos uma tabela estranha, de poucas linhas, chamada cd_categoria.

In [60]:
db("DESCRIBE CD_CATEGORIA")

Executando query:
('Codigo_Categoria', 'int', 'NO', '', None, '')
('Menor_Preco', 'int', 'NO', '', None, '')
('Maior_Preco', 'int', 'NO', '', None, '')


In [61]:
db("SELECT * from CD_CATEGORIA")

Executando query:
(1, 5, 10)
(2, 10, 12)
(3, 12, 15)
(4, 15, 20)


Esta tabela lista as categorias de preço de CDs. Por exemplo, se um CD custa 9 reais, ele está na categoria 1.

Gostariamos de usar essa informação em conjunto com a tabela de CDs para obter a lista de CDs em cada categoria. Por onde começar?

Vamos montar o produto cartesiano desta tabela cd_categoria com a tabela de CDs e ver se algum *insight* aparece:

In [62]:
db("SELECT * FROM CD, CD_CATEGORIA LIMIT 10")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 15, 20)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 3, 12, 15)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 2, 10, 12)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 1, 5, 10)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 4, 15, 20)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 12, 15)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 10, 12)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 1, 5, 10)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 4, 15, 20)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 12, 15)


Interessante! Como se trata do produto cartesiano, cada CD é listado repetidas vezes, uma para cada categoria de CDs. 

*Cada linha tem* ***o preço do CD*** *e um par de valores com* ***limites de preço de faixa***. Leia esta sentença quantas vezes for necessário.

E se filtrássemos esse resultado para manter apenas as linhas onde o preço do CD está dentro dos limites de preço?

In [63]:
db("""
SELECT 
    * 
FROM 
    CD c,
    CD_CATEGORIA cat 
WHERE 
    c.Preco_Venda BETWEEN cat.Menor_Preco AND cat.Maior_Preco
""")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 15, 20)
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 3, 12, 15)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 12, 15)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 2, 10, 12)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 12, 15)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1, 3, 12, 15)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2, 2, 10, 12)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 2, 10, 12)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 1, 5, 10)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None, 1, 5, 10)


Surgiu um pequeno problema: tem CD alocado para duas categorias ao mesmo tempo! Culpa do operador `BETWEEN` e da nossa interpretação da tabela `cd_categoria`. Vamos remover o `BETWEEN` e usar os limites do jeito que a gente quer:

In [64]:
db("""
SELECT 
    * 
FROM 
    CD cd, 
    CD_CATEGORIA cat 
WHERE 
    cd.Preco_Venda >= cat.Menor_Preco 
    AND cd.Preco_Venda < cat.Maior_Preco
""")

Executando query:
(1, 1, 'Mais do Mesmo', 15, datetime.date(1998, 10, 1), 5, 4, 15, 20)
(2, 2, 'Bate-Boca', 12, datetime.date(1999, 7, 1), 3, 3, 12, 15)
(3, 3, 'Elis Regina - Essa Mulher', 13, datetime.date(1989, 5, 1), 1, 3, 12, 15)
(4, 2, 'A Força que nunca Seca', 14, datetime.date(1998, 12, 1), 1, 3, 12, 15)
(5, 3, 'Perfil', 11, datetime.date(2001, 5, 1), 2, 2, 10, 12)
(6, 2, 'Barry Manilow Greatest Hits Vol I', 10, datetime.date(1991, 11, 1), 7, 2, 10, 12)
(7, 2, 'Listen Without Prejudice', 9, datetime.date(1991, 10, 1), None, 1, 5, 10)


Finalmente, vamos ajustar nossa query para buscar os nomes de CDs na faixa 2 de preço:

In [65]:
db("""
SELECT 
    Nome_CD 
FROM 
    CD cd, 
    CD_CATEGORIA cat 
WHERE 
    cd.Preco_Venda >= cat.Menor_Preco 
    AND cd.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_categoria = 2
""")

Executando query:
('Perfil',)
('Barry Manilow Greatest Hits Vol I',)


#### Vamos praticar

**Exercício 05**: Liste o nome do CD e de sua respectiva gravadora para CDs na categoria 2.

In [ ]:
db("""
-- SUA QUERY AQUI!
""")

In [66]:
sql_ex05 = """
SELECT 
    Nome_CD, Nome_Gravadora 
FROM 
    CD cd, 
    CD_CATEGORIA cat,
    GRAVADORA g
WHERE 
	cd.Codigo_Gravadora = g.Codigo_Gravadora
    AND cd.Preco_Venda >= cat.Menor_Preco 
    AND cd.Preco_Venda < cat.Maior_Preco
    AND cat.Codigo_categoria = 2;
"""

db(sql_ex05)

Executando query:
('Barry Manilow Greatest Hits Vol I', 'BMG')
('Perfil', 'SOM LIVRE')


In [67]:
ia.sender(answer="sql_ex05", task="agg_join", question="ex05", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex05', style=ButtonStyle()), Output()), _dom_classes=('widget…

Vamos agora fechar a conexão com este banco de dados, porque no próximo tópico vamos abrir uma conexão com outro banco de dados.

In [68]:
connection.close()

### Outer join

![](imgs/sql-outer-joins.png)

Fonte da imagem: https://www.dofactory.com/img/sql/sql-outer-joins.png

Observação: MySQL não suporta `FULL OUTER JOIN` nativamente, o que precisa ser feito utilizando `UNION`.

Considere a seguinte base de testes: cada perigo tem um nome e cada comida tem um nome e um perigo associado. (Use o script `tranqueira.sql` para testar essa base.)

```SQL
DROP DATABASE IF EXISTS tranqueira;
CREATE DATABASE tranqueira;
USE tranqueira;

CREATE TABLE comida (
	id INT NOT NULL AUTO_INCREMENT,
	Nome VARCHAR(30),
    idPerigo INT,
    PRIMARY KEY (id)
);

CREATE TABLE perigo (
	id INT NOT NULL AUTO_INCREMENT,
    Nome VARCHAR(20),
    PRIMARY KEY (id)
);

ALTER TABLE comida ADD FOREIGN KEY (idPerigo) REFERENCES perigo (id);

INSERT INTO perigo VALUES (1, 'Cardiaco'), (2, 'Intestinal'), (3, 'Dermatologico'), (4, 'Mental');
INSERT INTO comida VALUES (1, 'Torresmo', 1), (2, 'Alface', NULL), (3, 'Coxinha', 2), (4, 'Espetinho', 2);

SELECT * FROM comida;
SELECT * FROM perigo;
```

![Tabela comida](imgs/comida.png)

![Tabela perigo](imgs/perigo.png)

Fazendo o inner join da tabela comida com a tabela perigo temos a lista de comidas que estão associadas a algum perigo:

In [69]:
connection_tranqueira = mysql.connector.connect(
    host=os.getenv("MD_DB_SERVER"),
    user=os.getenv("MD_DB_USERNAME"),
    password=os.getenv("MD_DB_PASSWORD"),
    database="tranqueira"
)

dbt = partial(run_db_query, connection_tranqueira)

dbt("USE tranqueira")

dbt("""
SELECT 
    comida.Nome, perigo.Nome 
FROM 
    comida INNER JOIN perigo ON comida.idPerigo = perigo.id
""")

Executando query:
Executando query:
('Torresmo', 'Cardiaco')
('Coxinha', 'Intestinal')
('Espetinho', 'Intestinal')


Note que a alface não aparece aqui. 

Se quisermos que todas as comidas apareçam neste join, mesmo que não estejam associadas a um perigo, como fazer? Para isso servem os ***outer joins***. Vamos experimentar com o **`LEFT OUTER JOIN`**:

In [70]:
dbt("""
SELECT 
    comida.Nome, 
    perigo.Nome 
FROM
    comida
    LEFT OUTER JOIN perigo ON comida.idPerigo = perigo.id
""")

Executando query:
('Torresmo', 'Cardiaco')
('Alface', None)
('Coxinha', 'Intestinal')
('Espetinho', 'Intestinal')


Veja que agora podemos observar que a alface não oferece perigo! E se quisermos fazer o contrário: queremos que todos os perigos apareçam, mesmo que não exista comida associada? Vamos usar o **`RIGHT OUTER JOIN`**:

In [71]:
dbt("""
SELECT 
    comida.Nome,
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
""")

Executando query:
('Torresmo', 'Cardiaco')
('Espetinho', 'Intestinal')
('Coxinha', 'Intestinal')
(None, 'Dermatologico')
(None, 'Mental')


Agora os perigos dermatológico e mental aparecem, mas não existem comidas associadas a eles!

Os dois tipos de join são redundantes: basta inverter a ordem das partes para trocar de `RIGHT OUTER JOIN` para `LEFT OUTER JOIN`. Por exemplo:

In [72]:
dbt("""
SELECT
    comida.Nome,
    perigo.Nome
FROM
    perigo
    LEFT OUTER JOIN comida ON comida.idPerigo = perigo.id
""")

Executando query:
('Torresmo', 'Cardiaco')
('Espetinho', 'Intestinal')
('Coxinha', 'Intestinal')
(None, 'Dermatologico')
(None, 'Mental')


#### Vamos praticar

**Exercício 6**: Liste os perigos que não estão associados a nenhuma comida.

In [75]:
sql_ex06 = """
SELECT 
    perigo.Nome
FROM
    comida
    RIGHT OUTER JOIN perigo ON comida.idPerigo = perigo.id
WHERE
	comida.Nome IS NULL;
"""

dbt(sql_ex06)

Executando query:
('Dermatologico',)
('Mental',)


In [76]:
ia.sender(answer="sql_ex06", task="agg_join", question="ex06", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex06', style=ButtonStyle()), Output()), _dom_classes=('widget…

# Resumo

Representação dos joins mais utilizados em MySQL.

![](imgs/joins.png)

Fonte da imagem: https://user-images.githubusercontent.com/47733530/152173611-fe092793-7eb2-4b1a-a4ce-c0ed807c1b6e.png

## Finalizando o trabalho

Por fim, vamos fechar nossa conexão com o banco de dados!

In [77]:
connection.close()
connection_tranqueira.close()

Se você tentar fechar a conexão duas vezes, nada acontecerá nesta biblioteca. Outras bibliotecas podem lançar terá uma exceção: consulte a documentação da sua biblioteca.

In [78]:
connection.close()

Por hoje é só! Pratique com os exercícios do seu livro texto.

## Conferir Notas

Confira se as notas na atividade são as esperadas!

Primeiro na atividade atual!

In [79]:
ia.grades(by="TASK", task="agg_join")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | agg_join |     10 |

In [80]:
ia.grades(task="agg_join")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | agg_join    | ex01        |      1 |     10 |
|  1 | agg_join    | ex02        |      1 |     10 |
|  2 | agg_join    | ex03        |      1 |     10 |
|  3 | agg_join    | ex04        |      1 |     10 |
|  4 | agg_join    | ex05        |      1 |     10 |
|  5 | agg_join    | ex06        |      1 |     10 |

E agora em todas as tarefas!

In [81]:
ia.grades(by="TASK")

|    | Tarefa   |   Nota |
|---:|:---------|-------:|
|  0 | agg_join |     10 |
|  1 | ddl      |     10 |
|  2 | dml      |     10 |
|  3 | newborn  |     10 |
|  4 | select01 |     10 |

In [82]:
ia.grades()

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | agg_join    | ex01        |      1 |     10 |
|  1 | agg_join    | ex02        |      1 |     10 |
|  2 | agg_join    | ex03        |      1 |     10 |
|  3 | agg_join    | ex04        |      1 |     10 |
|  4 | agg_join    | ex05        |      1 |     10 |
|  5 | agg_join    | ex06        |      1 |     10 |
|  6 | ddl         | ex02        |      1 |     10 |
|  7 | ddl         | ex03        |      1 |     10 |
|  8 | ddl         | ex04        |      1 |     10 |
|  9 | ddl         | ex05        |      1 |     10 |
| 10 | ddl         | ex06        |      1 |     10 |
| 11 | ddl         | ex07        |      1 |     10 |
| 12 | ddl         | ex09        |      1 |     10 |
| 13 | ddl         | ex10        |      1 |     10 |
| 14 | ddl         | ex11        |      1 |     10 |
| 15 | dml         | ex01        |      1 |     10 |
| 16 | dml         | ex02        |      1 |     10 |
| 17 | dml         | ex03        |      1 |     10 |
| 18 | dml         | ex04        |      1 |     10 |
| 19 | dml         | ex05        |      1 |     10 |
| 20 | dml         | ex06        |      1 |     10 |
| 21 | newborn     | ex01        |      1 |     10 |
| 22 | select01    | ex01        |      1 |     10 |
| 23 | select01    | ex02        |      1 |     10 |
| 24 | select01    | ex03        |      1 |     10 |
| 25 | select01    | ex04        |      1 |     10 |
| 26 | select01    | ex05        |      1 |     10 |